# Lecture XIII: A simple 1D Particle-Mesh code


Aim: we want to integrate equations for the evolution of a collisionless fluid acting under the action of gravity.
The equations governing the system are the **Vlasov-Poisson** ones:

![Vlasov-Poisson](VlasovPoisson.png)


where the distribution function **f(x,v,t)** depends on position, velocities and time. The density is:
<font size="6">
$$
\rho(x,t) = \int f(x,v,t) dv
$$
</font>

Such equations (in 3D) lives in a 6D+1 dimensions, and their direct integration (using e.g. _Boltzmann codes_) is highly inefficient.

Thus, we can obtain the momentum equation by multiplying VP equations by v and integrating over velocities:
![Momentum equation](VlasovPoissonMoment.png)

where the _pressure tensor_ is:
![Pressure tensor](PressureTensorVP.png)

These are simply the equations of a compressible fluid supported by pressure in the form of velocity dispersion.
Choosing a particular solution, called _single speed_ because the velocities are a function of the positions (no multiple velocities at the same point in space:

![single speed](singlespeed.png)

VP equations become the _Euler-Poisson_ equations:

![Euler-Poisson](EuleroPoisson.png)

These are familiar, but becomes undefined at _shell crossing_ (where more fluid elements do cross) because the density becomes infinite.

Using instead an _isotropy ansatz_ for the pressure tensor:

![isotropy ansatz](IsotropicalPressure.png)

we simply introduce an equation of state of the kind
<font size="6">
$$
P = P(\rho)
$$
</font>

and obtain equations similar to those used for gases:

![EP with P](EuleroPoisson-withP.png)

but here P is not the hydrodinamical pressure.

If we sample the fluid with _massive particles_ (**N-Body codes**), evolve them under the action of gravity and allow them to cross, we obtain a system that is Vlasov_Poisson at all scales _above the scale that define the massive particle themselves_.

However, doing so we do not exacly resolve the VP equations! We work in configuration space, and don't explicitly follow the evolution of the momentum part of the phase space.

There are several numerical codes to deal with this problem, most diffuse ones being _direct codes_, _tree codes_, _particle-mesh codes_ and combinations of them.


A Particle-Mesh code uses massive particle to define a density field on a mesh, an solves the Poisson equation such a mesh. Then, it evaluates the forces on the mesh, interpolates them to particles, and drift particles.
This scheme is particularly fast because it solves the Poisson equation **in the Fourier space**, making use of a numerical algorithm called **Fast Fourier Transform** that is very efficient.


The Particle-Mesh scheme is thus:
 - the density field is estimated on a grid using _massive particles_, representing _fluid elements_.
 - the density field is transformed to the Fourier space
 - the gravitational potential is computed using the green function of the Laplacian
 - the gravitational potential is transformed back and forces are evaluated
 - forces (on the mesh) are interpolated to particle
 - particles velocities are updated with a chosen time-step
 - particles positions are then updated, usually with a leap-frog integrator
 - cycle is repeated until the desired final time.

We will write a simple, non-cosmological unidimensional PM code.

 
 

## Initial conditions

We need an *external* code to generate initial conditions (ICs) of the form x,v for N particles.
To do this, we need to **fix the units of measurements** (UoM).

Use a parameter file to set UdM and other relevant characteristics. We will use commonly used Udm:

UnitVel 1e5 (1 km/s)

UnitMass 1.989e43 (10^10 Msol)

UnitLength 3.085678e21 (1 kpc)

...compute the unit of time and the value of G!

Other needed parameters will be:

N_points   number of mass points

N_grid     FFT grid

BoxLenght  Length of the box (in kpc)

A_deltaPar Maximum density contrast


other useful values could also be added to the parameter file (e.g. H0, rho_crit...)

We will evolve a sinusoudal density contrast:
$$
\delta = A sin(x \cdot 2\pi/L  - \pi/2)
$$

A will be small because the initial density contrast must be linear. Velocities will be set to zero.

Note that using physical UoM means that we have a 3D distribution with only radial (1D) perturbations. Clearly, in reality this setup would not be stable against perturbations in the other two spherical coordinates.

The IC code should be able to write output in binary or in text, on request. It should also produce an histogram of the density to verify its correctness.


## Density computation

Once we have the (initial or evolved) particle distribution, we must coompute the density on a grid, to be able to perform the Fourier transform and evaluate the potential.

The simplest way is to simply assign the mass of each particle to the grid cell it belongs. This scheme is called **Nearest Grid Point** (NGP).

The problem is that the density field, using NGP, is discontinuos. One can then "smooth" the mass on the two nearest grid cells, proportionally to the distance of the particle to the grid cell center. This scheme is called **Cloud in Cell** (CIC) and gives a continuos density field but a discontinuos first derivative. 

The continuity can be assured for higher order derivatives using more cells. Assigning the mass on three cells is done with the **Triangular shaped cloud** (TSC) scheme.

Graphically:
![PM density assignment scheme](pm-assignment-schemes.jpg)


The density in each cell will be:
<font size="4">
$$
\rho_j = \sum_{i=1}^{N_p} m_i W(x_i - x_j)
$$
</font>
where $\rho_j$ is the density in the grid cell, $x_i$ the particle position, $N_p$ the particle number and $x_j$ the position of the center of the grid cell. Note that W(x) is zero if more than 1,2,3 cells are considered.

Clearly, the higher the order of the assignment scheme is, the more precise is the computation.

The function that evaluate densities should be able to use one of the above scheme, depending on one appropriate $define. For testing purpose, print out the produced densities and plot them.


## Fast Fourier Transform and potential computation

The numerical solution of the Poisson equation would require a **double integral**, that is not very stable and is unefficient.
By transforming the density in the Fourier space, the Laplacian operator becomes a simple multiplication:
<br><br>
<font size="4">
$$
V_k = G_k \delta_k = -1/k^2 \delta_k
$$
</font>
<br><br>

Actually $G_k=-1/k^2$ is called *poor man green function*!
This is because it is the Green function of the Laplace operator only for infinite continous domains. 
Keeping in consideration a finite box and the discretization of the space, 
<br><br>
<font size="4">
$$
G_k \propto [sin^2(k_x/2)]^{-1}
$$
</font>
<br><br>
...but we will use in a first approach the *poor man* one.

<br><br><br><br>
To implement the potential computation we will use the FFTW 3.x library. 
The basics are:

<br><br>
**definition and allocation of FFT data**

  include "fftw3.h"

  fftw_complex  \*kDensity, \*kPot;

  fftw_plan fft_real_fwd, fft_real_bck;
  
  kDensity  = fftw_alloc_complex( Ngrid );
  
  kPot  = fftw_alloc_complex( Ngrid );
  
  Pot     = (double*) malloc( Ngrid * sizeof(double) );

  Density = (double*) malloc( Ngrid * sizeof(double) );
  
  
<br><br>
**definition of back and fourth FFT:**

 fft_real_fwd = fftw_plan_dft_r2c_1d(Ngrid, Density, kDensity, FFTW_ESTIMATE );

 fft_real_bck = fftw_plan_dft_c2r_1d(Ngrid, kPot, Pot, FFTW_ESTIMATE );
 
<br><br>
**execution of direct FFT:**
 
  fftw_execute( fft_real_fwd );

<br><br>
**computation of the potential in the Fourier space:**

  norm = 2 * M_PI / BoxSize;

  for( int i=1; i<Ngrid/2+1; i++)

{

      k = (i*1.0) * norm;
      
      kPot[i][0] = -kDensity[i][0]/k/k;
      
      kPot[i][1] = -kDensity[i][1]/k/k;
      
}

Note that the Fourier transform of the density is complex...

Note the definition of the wave numbers.

<br><br>
**execution of the inverse FFT and normalization:**

  fftw_execute( fft_real_bck );

  /\* normalize  \*/  
  
  double       norm  = 1.0 / Ngrid;
  
  for( int i=0; i<Ngrid; i++)
  
    Pot[i] *= norm;

...at this point you have the gravitational potential, ready to be differentiated to get the force.

A more detailed treatment of FFTW will be given by Prof. Taffoni.

<br><br>
*When implementing this part, make tests to be sure that the machinery is working. This can be easily done since in 1D, the Poisson equation has an analitic solution for a number of forms for $\rho - \bar{\rho}$* !


## Force computation and interpolation



## Leap Frog

